In [3]:
"""CT_Lymph_Nodes dataset."""

import tensorflow_datasets.public_api as tfds
import tensorflow as tf
import numpy as np
import os
import io
import pydicom
import nibabel 



In [4]:
tfds.load('ct__lymph__nodes')

ValueError: Dtype float64 do not match float16

In [ ]:
!pwd

filepath = '/Users/Jingjing/tensorflow_datasets/downloads/manual'
def _generate_examples(filepath):
    """Yields examples for the CT lymph nodes dataset
    Args:
        filepath: path to the CT lymph nodes files
    Yields:
        Dictionaries with images and masks
    
    """
    ## Each patient has his own folder of masks and images, and the patient id is the same in masks and images
    patients = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_MASKS'))
    patients.sort()
    

    ## iterate over all masks folders
    mask_lst = []
    for patient_id in patients:
        try:
            mask = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_MASKS',patient_id))
            if mask[0].endswith('.nii.gz'):
                file_name = os.path.join(filepath,'MED_ABD_LYMPH_MASKS',patient_id,mask[0])
                mask_lst.append((patient_id,nibabel.load(file_name)))
        except:
            pass

    ## iterate over all images folders
    for patient_id in patients:
        try:
            mask_file = [item for item in mask_lst if item[0] == patient_id ][0][1]
            ## files are stored in sub-directories, so go into the sub-directory where stores the images
            first = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_IMAGES',patient_id))[0]
            second = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_IMAGES',patient_id,first))[0]
            third = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_IMAGES',patient_id,first,second))[0]
            file_name = os.path.join(filepath,'MED_ABD_LYMPH_IMAGES',patient_id,first,second,third)
            if file_name.endswith('dcm'):
                image_file = pydicom.read_file(file_name)
                yield file_name,
                {
                    'image':image_file.pixel_array,
                    'mask' : mask_file.get_fdata(),
                    'id' : image_file.PatientID,
                    'age' : image_file.PatientAge,
                    'sex' :image_file.PatientSex,
                    'body_part': image_file.BodyPartExamined

                }
        except:
            pass


In [ ]:
patient_id = 'ABD_LYMPH_001'
filepath = '/Users/Jingjing/tensorflow_datasets/downloads/manual/MED_ABD_LYMPH_IMAGES/ABD_LYMPH_001/09-14-2014-ABDLYMPH001-abdominallymphnodes-30274/abdominallymphnodes-26828/1-001.dcm'
pydicom.read_file(filepath).pixel_array.dtype




In [ ]:
filepath = '/Users/Jingjing/tensorflow_datasets/downloads/manual/MED_ABD_LYMPH_MASKS/ABD_LYMPH_001/ABD_LYMPH_001_mask.nii.gz'
import numpy as np
nibabel.load(filepath).shape

In [ ]:
    filepath = '/Users/Jingjing/tensorflow_datasets/downloads/manual'
    patients = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_MASKS'))
    patients.sort()
    
    
    ## iterate over all masks folders
    mask_lst = []
    for patient_id in patients:
        try:
            mask = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_MASKS',patient_id))
            for file in mask:
                
                if file.endswith('.nii.gz'):
                    file_name = os.path.join(filepath,'MED_ABD_LYMPH_MASKS',patient_id,mask[0])
                    mask_lst.append((patient_id,nibabel.load(file_name)))
        except:
            pass

    ## iterate over all images folders
    patient_id='ABD_LYMPH_002'
    mask_file = [item for item in mask_lst if item[0] == patient_id ][0][1].get_fdata()
    first = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_IMAGES',patient_id))[0]
    second = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_IMAGES',patient_id,first))[0]
    third = tf.io.gfile.listdir(os.path.join(filepath,'MED_ABD_LYMPH_IMAGES',patient_id,first,second))
    third.sort()
    i = 0
    for file in third:
            file_name = os.path.join(filepath,'MED_ABD_LYMPH_IMAGES',patient_id,first,second,file)
            if file_name.endswith('dcm'):
                key = patient_id+'_'+str(i)
                image_file = pydicom.read_file(file_name)
                print( key,
                        {
                            'image':image_file.pixel_array,
                            'mask' : mask_file[:,:,i],
                            'age' : image_file.PatientAge,
                            'sex' :image_file.PatientSex,
                            'body_part': image_file.BodyPartExamined

                        })
            i+=1
    

In [ ]:
tfds.load('ct__lymph__nodes')

In [ ]:
from tensorflow_datasets.core import SplitGenerator
builder = CT_Lymph_Nodes()
test_generator = SplitGenerator(name='test', gen_kwargs={"filepath": filepath})

In [ ]:
builder._split_generators = lambda _: [test_generator]

In [ ]:
builder.download_and_prepare()

In [ ]:
_masks_url = """https://wiki.cancerimagingarchive.net/download/attachments/19726546/MED_ABD_LYMPH_MASKS.zip?version=1&modificationDate=1449684916503&api=v2
"""